In [ ]:
import os
os.chdir('/home/extra/micheal/dd_synthesis')

# Segmentation

The goal of the segmentation network in DD Synthesis paper is to identify the face mask of both source and the target domain.

## Transform the Line Segmentation to Area Segmentation

The first chanllenge is that, using the original approach, we need to find out the "mask" of the feature to be integrated. In the OCT datasets, the segmentations are given as line layers. We have to transform the lines to area masks.

### 1. Visualize the Current Segmentations

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

def visualize_segmentation(bscan, label, show_original=False, alpha=0.8):
    color_map = {
        1: 'white',
        2: 'red',
        3: 'gray',
        4: 'orange',
        6: 'gainsboro'
    }
    if show_original:
        fig, axs = plt.subplots(1, 2, figsize=(5, 10))
        axs[0].imshow(bscan)
        axs[0].axis('off')
        axs[0].set_title('original')
        axs[1].set_title('segmentation')
        draw_handle = axs[1]
    else:
        draw_handle = plt
    draw_handle.axis('off')
    draw_handle.imshow(bscan)
    for k, color in color_map.items():
        x, y = np.where(label==k)
        draw_handle.scatter(y, x, color=color, alpha=alpha, linewidths=0, s=0.5)

In [ ]:
def get_bscan_label(img_name, bscan_prefix="data/ioct/bscans/val/", label_prefix="data/ioct/labels/val/"):
    bscan = Image.open(bscan_prefix + img_name)
    label = np.asarray(Image.open(label_prefix + img_name))
    return bscan, label

In [ ]:
bscan_prefix = "data/ioct/bscans/val/"
label_prefix = "data/ioct/labels/val/"
img_name = "5d396575-e039-49da-a219-fe239e8bd9c88062-101.png"

bscan, label = get_bscan_label(img_name, bscan_prefix, label_prefix)
visualize_segmentation(bscan, label, show_original=True)

In [ ]:
img_name = "OS-2020-02-03_114037fs-039.png"
bscan, label = get_bscan_label(img_name)
visualize_segmentation(bscan, label, show_original=True)

In [ ]:
img_name = "OS-2020-02-03_135535fs-089.png"
bscan, label = get_bscan_label(img_name)
visualize_segmentation(bscan, label, show_original=True)

In [ ]:
def get_shadow(label):
    shadow_x = np.array([], dtype=np.int64)
    shadow_y = np.array([], dtype=np.int64)
    # Requirements for the shadow label:
    # 1. Horizontally after the starting of the instrument/mirroring & before the
    #    ending of the instrument/mirroring
    # 2. Vertically below the (upperbound of) label 1
    x, y = np.where(np.logical_or(label==2, label==4)) # (1024, 512)
    if len(x) == 0:
        return shadow_x, shadow_y
    left_bound = np.min(y)
    right_bound = np.max(y)
    x, y = np.where(label==1)
    upper_bound = np.min(x)
    left_end = upper_bound
    right_end = upper_bound
    for i in (left_bound, 0, -1):
        left_1 = np.where(label[:, i]==1)[0]
        if len(left_1) > 0:
            left_end = left_1[0]
            break
    for i in range(right_bound, 512):
        right_1 = np.where(label[:, i]==1)[0]
        if len(right_1) > 0:
            right_end = right_1[0]
            break
    upper_bound = max(upper_bound, min(left_end, right_end))
    for i in range(left_bound, right_bound):
        x_vertical = np.arange(upper_bound, 1024) # upperbound to bottom
        y_vertical = np.full_like(x_vertical, i)
        shadow_x = np.concatenate([shadow_x, x_vertical])
        shadow_y = np.concatenate([shadow_y, y_vertical])
    return shadow_x, shadow_y

In [ ]:
x, y = get_shadow(label)
label_copy = label.copy()
label_copy[x, y] = 6
visualize_segmentation(bscan, expand_label(label_copy), show_original=True, alpha=0.1)

### 2. Expand Label to Cover the Whole Area

By observing the bscans, we found out we need to:
1. Expand instrument & mirroring labels horizontally
2. Invent a new shadow label which reside below the instruments and mirroring.

Note: for the shadow label, we don't really wish it to resembel the target domain (iOCT). We wish it to keep the noise texture of the original OCT domain, but leaves blank.

In [ ]:
color_map = {
    1: 'white',
    2: 'red',
    3: 'gray',
    4: 'orange',
    6: 'gainsboro'
}

def horizontal_expand(label, feature, to_expand=20):
    label_copy = label.copy()
    x, y = np.where(label_copy==feature)
    xacc, yacc = x, y
    for i in range(to_expand):
        xacc = np.concatenate([xacc, x])
        yacc = np.concatenate([yacc, y+i])
    label_copy[xacc, yacc] = feature
    return label_copy

def expand_label(label, expansion2=30, expansion4=60):
    # For label 2 4 (instrument & its mirroring), we horizontally expand
    # a couple of pixels rightward
    label = horizontal_expand(label, 2, to_expand=expansion2)
    label = horizontal_expand(label, 4, to_expand=expansion4) # shadows are generally broader
    return label

img_name = "OS-2020-02-03_114037fs-039.png"
bscan, label = get_bscan_label(img_name)
visualize_segmentation(bscan, expand_label(label), show_original=True)

In [ ]:
x, y = get_shadow(label)
label_copy = label.copy()
label_copy[x, y] = 6
visualize_segmentation(bscan, expand_label(label_copy), show_original=True, alpha=0.1)

## Train a Segmentation Network

The strategy is to stick to the original labels for classification. After we get the segmentation as lines, we expand it to area masks using the methods above.

The original paper use a repurpose GAN as segmentation network with one-shot learning. But I don't see the advantage of using it over a plain segmenter. (Or probably the paper explained it? **#CHECK**)

Here I use a simple U-Net structure as the segmentation network.

### Update: Seems No Need

The masks are given as ground truth, maybe we don't have to train a new segmentor for the task.